In [ ]:
# Gemini - Fred 11/Oct 12h12 pm - corrections applied (USE THIS ONE!)
# Interleaved coherence - 1000 loop
# For q0, 100 iter, 1024 reps per iteration takes 1h 15 min
qubit=q4 # CHANGE
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import csv
import gc # Import garbage collector

# Constants moved outside the loop
T1 = 20e-6 # estimation of T1 for qubit - change this based on the T1 results
ramsey_est = 16e-6
T2E_est = 13e-6

quantum_device.cfg_sched_repetitions(1000) # CHANGE
points = 30 
iterations=1000 # CHANGE

with open('20251022_night_coherence_monitoring_log_q4.csv', mode='w', newline='') as file: # CHANGE
    writer = csv.writer(file)
    writer.writerow(['Iteration', 'Timestamp', 'T1', 'T1_std', 'T2*', 'T2*_std', 'T2E', 'T2E_std', 'Qubit Frequency'])

    # Main loop
    for i in range(iterations):
        
        # --- Memory Optimization: Define time arrays INSIDE the loop ---
        # Defining them here ensures they are garbage collected after each iteration.
        t1_times = np.linspace(qubit.rxy.duration()*1.5, 4*T1, points)
        t1_times = t1_times - t1_times%4e-9

        ramsey_times = np.linspace(qubit.rxy.duration()*3, 4*ramsey_est, points)
        ramsey_times = ramsey_times - ramsey_times%8e-9

        echo_times = np.linspace(qubit.rxy.duration()*3, 4*T2E_est, points)
        echo_times = echo_times - echo_times%8e-9

        
        # Print the current iteration number 🌟
        print(f"Running iteration {i + 1} of {iterations}")
        
        # Timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Measure coherence
        try:
            analyzed_dset = meas.single_qubit_timedomain.measure_coherence_times(
                quantum_device=quantum_device,
                qubit_name="q4",
                t1_times=t1_times,
                ramsey_times=ramsey_times,
                echo_times=echo_times,
                artificial_detuning=3 / ramsey_times[-1],
                plot_figures=False
            )
        
            # Extract T1
            try:
                t1 = analyzed_dset.t1_analysis.quantities_of_interest['T1'].nominal_value
                t1_std_dev = analyzed_dset.t1_analysis.quantities_of_interest['T1'].std_dev
            except:
                t1 = np.nan
                t1_std_dev = np.nan
        
            # Extract T2* and qubit frequency
            try:
                if 'T2*' in analyzed_dset.ramsey_analysis.quantities_of_interest:
                    t2_star = analyzed_dset.ramsey_analysis.quantities_of_interest['T2*'].nominal_value
                    t2_star_std_dev = analyzed_dset.ramsey_analysis.quantities_of_interest['T2*'].std_dev
                    qubit_freq = analyzed_dset.ramsey_analysis.quantities_of_interest["qubit_frequency"].nominal_value
                elif 'T2*_1' in analyzed_dset.ramsey_analysis.quantities_of_interest:
                    t2_star = min(
                        analyzed_dset.ramsey_analysis.quantities_of_interest['T2*_1'].nominal_value,
                        analyzed_dset.ramsey_analysis.quantities_of_interest['T2*_2'].nominal_value
                    )
                    t2_star_std_dev = min(
                        analyzed_dset.ramsey_analysis.quantities_of_interest['T2*_1'].std_dev,
                        analyzed_dset.ramsey_analysis.quantities_of_interest['T2*_2'].std_dev
                    )
                    qubit_freq = analyzed_dset.ramsey_analysis.quantities_of_interest["qubit_frequency"].nominal_value
                else:
                    t2_star = np.nan
                    t2_star_std_dev = np.nan
                    qubit_freq = np.nan
            except:
                t2_star = np.nan
                t2_star_std_dev = np.nan
                qubit_freq = np.nan
        
            # Extract T2E
            try:
                t2e = analyzed_dset.echo_analysis.quantities_of_interest['t2_echo'].nominal_value
                t2e_std_dev = analyzed_dset.echo_analysis.quantities_of_interest['t2_echo'].std_dev
            except:
                t2e = np.nan
                t2e_std_dev = np.nan
            
            # --- Aggressive Cleanup Round 1 ---
            # Explicitly clear the large data object after processing
            del analyzed_dset 
            #meas.clear_cache()
            gc.collect() # Initial cleanup

        except Exception as e:
            print(f"Iteration {i+1}: failed with error {e}. Skipping data logging for this iteration.")
            t1, t1_std_dev, t2_star, t2_star_std_dev, t2e, t2e_std_dev, qubit_freq = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

        # Print iteration summary
        print(f"[{timestamp}] T1 = {t1:.2e}, T2* = {t2_star:.2e}, T2E = {t2e:.2e}, qubitfreq = {qubit_freq}")
    
        # Keep in CSV (This is the only place the data is stored permanently)
        writer.writerow([i + 1, timestamp, t1, t1_std_dev, t2_star, t2_star_std_dev, t2e, t2e_std_dev, qubit_freq])
        
        # Memory Optimization: Flush the write buffer to disk 💾
        file.flush()
        
        # --- Aggressive Cleanup Round 2 ---
        # Delete time arrays, which are recreated next iteration
        del t1_times
        del ramsey_times
        del echo_times
        gc.collect() # Second, aggressive cleanup